# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
import tarfile
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')
print(y_train.shape, 'train target')
print(y_test.shape, 'test target')

(50000, 32, 32, 3) train samples
(10000, 32, 32, 3) test samples
(50000, 1) train target
(10000, 1) test target


In [3]:
x_train_lt5 = x_train[y_train[:,0] < 5]
y_train_lt5 = y_train[y_train[:,0] < 5]
x_test_lt5 = x_test[y_test[:,0] < 5]
y_test_lt5 = y_test[y_test[:,0] < 5]

print(x_train_lt5.shape, 'train_lt5 samples')
print(x_test_lt5.shape, 'test_lt5 samples')
print(y_train_lt5.shape, 'train_lt5 target')
print(y_test_lt5.shape, 'test_lt5 target')

(25000, 32, 32, 3) train_lt5 samples
(5000, 32, 32, 3) test_lt5 samples
(25000, 1) train_lt5 target
(5000, 1) test_lt5 target


In [4]:
x_train_gt5 = x_train[y_train[:,0] >= 5]
y_train_gt5 = y_train[y_train[:,0] >= 5] - 5  # make classes start at 0 for
x_test_gt5 = x_test[y_test[:,0] >= 5]         # np_utils.to_categorical
y_test_gt5 = y_test[y_test[:,0] >= 5] - 5

print(x_train_gt5.shape, 'train_gt5 samples')
print(x_test_gt5.shape, 'test_gt5 samples')
print(y_train_gt5.shape, 'train_gt5 target')
print(y_test_gt5.shape, 'test_gt5 target')

(25000, 32, 32, 3) train_gt5 samples
(5000, 32, 32, 3) test_gt5 samples
(25000, 1) train_gt5 target
(5000, 1) test_gt5 target


### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_l5 = np_utils.to_categorical(y_train_lt5, 5)
y_test_l5 = np_utils.to_categorical(y_test_lt5, 5)

y_train_g5 = np_utils.to_categorical(y_train_gt5, 5)
y_test_g5 = np_utils.to_categorical(y_test_gt5, 5)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
epoch = 20
batch = 100

In [7]:
model = Sequential()

model.add(BatchNormalization(input_shape = (32,32,3)))
model.add(Convolution2D(32, (3,3), activation ='relu', input_shape = (32, 32, 3))) 
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Convolution2D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Convolution2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))
model.add(Convolution2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))


model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=32,activation = 'relu'))

model.add(Dense(units = 64, activation = 'relu'))

model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu'))

model.add(Dropout(0.2))
model.add(Dense(units = 5, activation = 'softmax')) 

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [8]:
model.fit(x_train_lt5, y_train_l5, batch_size=batch, nb_epoch=epoch, validation_data=(x_test_lt5, y_test_l5), callbacks=callback_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.




Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 124s 5ms/step - loss: 1.3314 - acc: 0.4022 - val_loss: 1.0084 - val_acc: 0.5914
Epoch 2/20
25000/25000 [==============================] - 123s 5ms/step - loss: 1.0119 - acc: 0.5819 - val_loss: 0.8835 - val_acc: 0.6416
Epoch 3/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.8941 - acc: 0.6449 - val_loss: 0.8010 - val_acc: 0.6840
Epoch 4/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.8224 - acc: 0.6846 - val_loss: 0.7043 - val_acc: 0.7330
Epoch 5/20
25000/25000 [==============================] - 123s 5ms/step - loss: 0.7615 - acc: 0.7130 - val_loss: 0.6810 - val_acc: 0.7464
Epoch 6/20
25000/25000 [==============================] - 123s 5ms/step - loss: 0.7109 - acc: 0.7364 - val_loss: 0.6167 - val_acc: 0.7728
Epoch 7/20
25000/25000 [==============================] - 123s 5ms/step - loss: 0.6765 - acc: 0.7505 - val_loss: 0.6110 -

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
model.save('/content/drive/My Drive/Colab Notebooks/nn_cifar10.h5')
model.save_weights('/content/drive/My Drive/Colab Notebooks/nn_cifar10_weights.h5')

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [11]:
for layers in model.layers:
    print(layers.name)
    if('dense' not in layers.name):
        layers.trainable = False
        print(layers.name + 'is not trainable\n')
    if('dense' in layers.name):
        print(layers.name + ' is trainable\n')

batch_normalization_1
batch_normalization_1is not trainable

conv2d_1
conv2d_1is not trainable

max_pooling2d_1
max_pooling2d_1is not trainable

dropout_1
dropout_1is not trainable

conv2d_2
conv2d_2is not trainable

max_pooling2d_2
max_pooling2d_2is not trainable

dropout_2
dropout_2is not trainable

conv2d_3
conv2d_3is not trainable

max_pooling2d_3
max_pooling2d_3is not trainable

dropout_3
dropout_3is not trainable

conv2d_4
conv2d_4is not trainable

max_pooling2d_4
max_pooling2d_4is not trainable

flatten_1
flatten_1is not trainable

dense_1
dense_1 is trainable

dense_2
dense_2 is trainable

dropout_4
dropout_4is not trainable

dense_3
dense_3 is trainable

dropout_5
dropout_5is not trainable

dense_4
dense_4 is trainable



### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [12]:
model.fit(x_train_gt5, y_train_g5, batch_size=batch, nb_epoch=epoch, validation_data=(x_test_gt5, y_test_g5), callbacks=callback_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 122s 5ms/step - loss: 1.1697 - acc: 0.5412 - val_loss: 0.6751 - val_acc: 0.7688
Epoch 2/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.7188 - acc: 0.7483 - val_loss: 0.5724 - val_acc: 0.8012
Epoch 3/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.6052 - acc: 0.7918 - val_loss: 0.4788 - val_acc: 0.8336
Epoch 4/20
25000/25000 [==============================] - 123s 5ms/step - loss: 0.5365 - acc: 0.8138 - val_loss: 0.4217 - val_acc: 0.8490
Epoch 5/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.5052 - acc: 0.8260 - val_loss: 0.3808 - val_acc: 0.8646
Epoch 6/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.4632 - acc: 0.8396 - val_loss: 0.3933 - val_acc: 0.8590
Epoch 7/20
25000/25000 [==============================] - 122s 5ms/step - loss: 0.4372 - acc: 0.8502 - val_loss: 0.3314 - v

## Sentiment analysis <br> 

The objective of the second problem is to perform Sentiment analysis from the tweets data collected from the users targeted at various mobile devices.
Based on the tweet posted by a user (text), we will classify if the sentiment of the user targeted at a particular mobile device is positive or not.

### 6. Read the dataset (tweets.csv) and drop the NA's while reading the dataset

In [0]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/tweets.csv', encoding = "ISO-8859-1").dropna()

In [15]:
data.shape

(3291, 3)

In [16]:
data.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


### Consider only rows having Positive emotion and Negative emotion and remove other rows from the dataframe.

In [0]:
data = data[(data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Positive emotion') | (data['is_there_an_emotion_directed_at_a_brand_or_product'] == 'Negative emotion')]

In [18]:
data.shape

(3191, 3)

### 7. Represent text as numerical data using `CountVectorizer` and get the document term frequency matrix

#### Use `vect` as the variable name for initialising CountVectorizer.

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [0]:
vect = CountVectorizer()

### 8. Find number of different words in vocabulary

In [0]:
words = vect.fit_transform(data.tweet_text)

In [39]:
print('Number of words: ', words.shape[1])

Number of words:  5648


#### Tip: To see all available functions for an Object use dir

In [40]:
#dir(cv)
dir(words)

['__abs__',
 '__add__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__idiv__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__pow__',
 '__radd__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmatmul__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '_add_dense',
 '_add_sparse',
 '_arg_min_or_max',
 '_arg_min_or_max_axis',
 '_asindices',
 '_binopt',
 '_cs_matrix__get_has_canonical_format',
 '_cs_matrix__get_sorted',
 '_cs_matrix__set_has_canonical_format',
 '_cs_matrix__set_sorted

### Find out how many Positive and Negative emotions are there.

Hint: Use value_counts on that column

In [41]:
pd.value_counts(data['is_there_an_emotion_directed_at_a_brand_or_product'])

Positive emotion    2672
Negative emotion     519
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

###  Change the labels for Positive and Negative emotions as 1 and 0 respectively and store in a different column in the same dataframe named 'label'

Hint: use map on that column and give labels

In [0]:
data['label'] = data.is_there_an_emotion_directed_at_a_brand_or_product.map({'Positive emotion':1, 'Negative emotion':0})

### 9. Define the feature set (independent variable or X) to be `text` column and `labels` as target (or dependent variable)  and divide into train and test datasets

In [0]:
X = data.tweet_text

In [0]:
y = data['is_there_an_emotion_directed_at_a_brand_or_product']

In [0]:
tweet_train,tweet_test,senti_train,senti_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [0]:
vecto = CountVectorizer()
tweet_train_dtm = vecto.fit_transform(tweet_train)
tweet_test_dtm = vecto.transform(tweet_test)

## 10. **Predicting the sentiment:**


### Use Naive Bayes and Logistic Regression and their accuracy scores for predicting the sentiment of the given text

In [58]:
# Logistic Regression
#Logistic Regression
logreg = LogisticRegression()
logreg.fit(tweet_train_dtm, senti_train)

y_pred_class = logreg.predict(tweet_test_dtm)
print('\nTest Accuracy Logistic Regression: ', metrics.accuracy_score(senti_test, y_pred_class))

y_pred_train = logreg.predict(tweet_train_dtm)
print('Train Accuracy Logistic Regression: ', metrics.accuracy_score(senti_train, y_pred_train))


Test Accuracy Logistic Regression:  0.863849765258216
Train Accuracy Logistic Regression:  0.9804075235109718


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [59]:
nb = MultinomialNB()
nb.fit(tweet_train_dtm, senti_train)
y_pred_class_nb = nb.predict(tweet_test_dtm)
print('Test Accuracy NB: ', metrics.accuracy_score(senti_test, y_pred_class_nb))
y_pred_train_nb = nb.predict(tweet_train_dtm)
print('Train Accuracy NB: ', metrics.accuracy_score(senti_train, y_pred_train_nb))

Test Accuracy NB:  0.865414710485133
Train Accuracy NB:  0.9521943573667712


## 11. Create a function called `tokenize_predict` which can take count vectorizer object as input and prints the accuracy for x (text) and y (labels)

In [0]:
def tokenize_test(vect):
    tweet_train_dtm = vect.fit_transform(tweet_train)
    tweet_test_dtm = vect.transform(tweet_test)
    print('Features: ', tweet_train_dtm.shape[1])
    nb = MultinomialNB()
    nb.fit(tweet_train_dtm, senti_train)
    y_pred_class = nb.predict(tweet_test_dtm)
    print('Accuracy: ', metrics.accuracy_score(senti_test, y_pred_class))

### Create a count vectorizer function which includes n_grams = 1,2  and pass it to tokenize_predict function to print the accuracy score

In [65]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test(vect)

Features:  25638
Accuracy:  0.8779342723004695


### 12. Create a count vectorizer function with stopwords = 'english'  and pass it to tokenize_predict function to print the accuracy score

In [66]:
vect = CountVectorizer(stop_words='english')
tokenize_test(vect)

Features:  4766
Accuracy:  0.8685446009389671


### 13. Create a count vectorizer function with stopwords = 'english' and max_features =300  and pass it to tokenize_predict function to print the accuracy score

In [67]:
vect = CountVectorizer(stop_words='english', max_features=300)
tokenize_test(vect)

Features:  300
Accuracy:  0.8215962441314554


### 14. Create a count vectorizer function with n_grams = 1,2  and max_features = 15000  and pass it to tokenize_predict function to print the accuracy score

In [68]:
vect = CountVectorizer(stop_words='english', max_features=15000)
tokenize_test(vect)

Features:  4766
Accuracy:  0.8685446009389671


### 15. Create a count vectorizer function with n_grams = 1,2  and include terms that appear at least 2 times (min_df = 2)  and pass it to tokenize_predict function to print the accuracy score

In [72]:
vect = CountVectorizer(ngram_range=(1,2), min_df=2)
tokenize_test(vect)

Features:  8179
Accuracy:  0.8575899843505478
